In [7]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
# import ROOT

In [8]:
%matplotlib inline
#fname = "coffeaOutput/trijetHistsTest_QCDsim_pt200.0_eta2.4_bbloose.pkl"
fname ="coffeaOutput/trijetHistsTest_JetHT_pt200.0_eta2.4_bbloose.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )

rootfile = uproot.recreate('trijetHistsJetHT.root')
#result=result[0]
# integrate and sum over axes
if "QCD" in fname:
    rootfile = uproot.recreate('rootFiles/trijetHistsQCDsim.root')
    rootfile['mreco_mgen_u'] = result['response_matrix_u'].project("mreco", "mgen")
    rootfile['mreco_mgen_g;1'] = result['response_matrix_g'].project("mreco","mgen")
    rootfile['ptreco_ptgen_u'] = result['response_matrix_u'].project("ptreco", "ptgen")
    rootfile['ptreco_ptgen_g'] = result['response_matrix_g'].project("ptreco","ptgen")
    rootfile['misses_ptgen_mgen'] = result['misses'].project("ptgen", "mgen")
    rootfile['fakes_ptreco_mreco'] = result['fakes'].project("ptreco","mreco")
    response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'].project("ptreco", "mreco", "ptgen", "mgen").to_numpy()
    response_matrix_g_values = result['response_matrix_g'].project("ptreco", "mreco", "ptgen", "mgen").values()
    print(response_matrix_g_values.shape)
    ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
    print(ptreco_centers.shape)
    mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
    ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
    mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
    print(np.shape(response_matrix_g_values))
    print(np.shape(response_matrix_g_values[np.newaxis]))
    rootfile['response'] = {'groomed':response_matrix_g_values[np.newaxis],
                       'ungroomed':response_matrix_u_values[np.newaxis]}
    rootfile['centers'] = {'ptreco':ptreco_centers[np.newaxis], 'mreco':mreco_centers[np.newaxis], 
                      'ptgen':ptgen_centers[np.newaxis], 'mgen':mgen_centers[np.newaxis]}
rootfile['jet_pt_mass_reco_u'] = result['jet_pt_mass_reco_u'][{'dataset':sum}]
rootfile['jet_pt_mass_reco_g'] = result['jet_pt_mass_reco_g'][{'dataset':sum}]
rootfile.close()

In [9]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [10]:
# print(fname["Events"].keys("*HTXS*"))

In [11]:
hists = uproot.open('trijetHistsJetHT.root')
print(hists['jet_pt_mass_reco_u'].to_numpy())


(array([[1.0000e+00, 0.0000e+00, 9.9300e+02, 1.6724e+04, 2.3478e+04,
        8.9880e+03, 5.5700e+03, 1.0160e+03, 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 1.3400e+02, 3.6000e+03, 2.6960e+03,
        1.1650e+03, 1.0540e+03, 9.1800e+02, 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 2.0000e+00, 9.3700e+02, 4.8800e+02,
        6.0000e+02, 4.3900e+02, 3.4400e+02, 4.0000e+01, 1.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 5.7000e+01, 1.8700e+02,
        5.6000e+01, 7.6000e+01, 1.1400e+02, 7.3000e+01, 1.6000e+01,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 1.0000e+00, 6.0000e+00, 6.0000e+01,
        4.3000e+01, 4.1000e+01, 5.4000e+01, 2.8000e+01, 1.2000e+01,
        1.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 3.0000e+00, 1.0000e+01,
        8.0000e+00, 3.0000e+00, 1.5000e+01, 1.0000e+01, 7.0000e+00,
        1.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6